In [1]:
import pandas as pd
import json
import imantics
import os.path as osp
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mammo_kp_dir = '/Users/lpires/Developer/dl/mammo_kp'

In [3]:
# read mount data

mount_df = pd.read_csv('mount_df.csv')
left_df = mount_df.rename(columns={'Lcase_id': 'case_id', 'Lline': 'line'})
left_df['laterality'] = 'L'
right_df = mount_df.rename(columns={'Rcase_id': 'case_id', 'Rline': 'line'})
right_df['laterality'] = 'R'
cases_df = pd.concat([left_df, right_df], sort=True)[['case_id', 'line', 'laterality']]
cases_df['line'] = cases_df['line'].apply(lambda s: json.loads(s))
cases_df.set_index('case_id', inplace=True)

In [4]:
# add keypoints to imantics

class CategoryKP(imantics.Category):
    def __init__(self, name, keypoints, skeleton, **kws):
        self.keypoints = keypoints
        self.skeleton = skeleton
        super().__init__(name, **kws)

    def coco(self, include=True):
        result = super().coco(include)
        result['keypoints'] = self.keypoints
        result['skeleton'] = self.skeleton
        return result

class AnnotationKP(imantics.Annotation):
    @classmethod
    def from_mask(cls, mask, keypoints, **kws):
        ann = super().from_mask(mask, **kws)
        ann.keypoints = keypoints
        return ann

    def coco(self, include=True):
        result = super().coco(include)
        result['num_keypoints'] = len(self.keypoints)
        result['keypoints'] = [value for sublist in self.keypoints for value in sublist]
        return result

In [5]:
# prepare imantics

dataset = imantics.Dataset('mammo kp')
category_breast = CategoryKP('breast', ['nipple', 'ending'], [1, 2], id=0)
category_pectoral_muscle = imantics.Category('pectoral_muscle', id=1)

In [6]:
# populate dataset

total = len(cases_df)
for i, (case_id, r) in enumerate(cases_df.iterrows()):
    print('\r%d/%d' % (i + 1, total), end='')
    if i > 1:
        continue
    line = r['line']
    # image
    img_fn = osp.join(mammo_kp_dir, '%d.png' % case_id)
    img = imantics.Image.from_path(img_fn)
    img.id = case_id
    # breasts
    breast_fn = osp.join(mammo_kp_dir, 'annotations', '%d_pixels0.png' % case_id)
    breast_img = cv2.imread(breast_fn)
    breast_img = cv2.cvtColor(breast_img, cv2.COLOR_BGR2GRAY)
    breask_mask = imantics.Mask.create(breast_img)
    line = r['line']
    nipple = [line[0], line[1], 2]
    ending = [line[2], line[3], 2]
    breask_ann = AnnotationKP.from_mask(breask_mask, [nipple, ending], image=img, category=category_breast)
    # pectoral muscle
    pectoral_muscle_fn = osp.join(mammo_kp_dir, 'annotations', '%d_pixels1.png' % case_id)
    pectoral_muscle_img = cv2.imread(pectoral_muscle_fn)
    pectoral_muscle_img = cv2.cvtColor(pectoral_muscle_img, cv2.COLOR_BGR2GRAY)
    pectoral_muscle_mask = imantics.Mask.create(pectoral_muscle_img)
    pectoral_muscle_ann = AnnotationKP.from_mask(pectoral_muscle_mask, [], image=img, category=category_pectoral_muscle)
    # add to image
    img.add(breask_ann)
    img.add(pectoral_muscle_ann)
    # add to dataset
    dataset.add(img)
print('\radded %d images' % total)

added 158 images


In [7]:
# plt.imshow(dataset.images[20588072].draw())
# plt.show()

In [8]:
dataset.save('mammo_kp.json')